In [1]:
import numpy as np
import matplotlib as plt
import pandas as pd
full_corpus = pd.read_csv('carteggio_svevo3.csv', sep=';')
ita_corpus = full_corpus[full_corpus['mainLanguage'] == 'ITA'].reset_index()

In [2]:
import nltk
#nltk.download('punkt')

Following the text of the project proposal the goal is to find:
- which are the main topics of discussion in the corpus, who are the persons which each topic is more associated with, how does the interest on different topics evolve over the time

In [3]:
#tokenization
t1 = ita_corpus['text'][0]

#all to lower case
t1 = t1.lower()


def tokenize_text(full_text):
    from nltk.tokenize import sent_tokenize, word_tokenize
    t_sent = sent_tokenize(full_text)
    t_words = []
    tk_sentence = []
    for sent in t1_sent:
        tk_sentence = ( word_tokenize(sent))
        for w in tk_sentence:
            t_words.append(w)

    return t_words

def adv_tokenizer(full_text):
    from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer('\w+')
    return tokenizer.tokenize(full_text)



def remove_stop_words(tokenized_text, stop_words):
    polished_text = [word for word in tokenized_text if not word in stop_words]
    return polished_text
    
#t1_t

In [4]:
#getting italian stop words
from stop_words import get_stop_words
it_stop_words = get_stop_words('it')

triestine_stop_words = ['xe', 'el']
it_stop_words.append(triestine_stop_words)
#triestine stop words? ['xe', 'el']
#tokenization
t1 = ita_corpus['text'][0]

#all to lower case
t1 = t1.lower()

t1 = adv_tokenizer(t1)
t1 = remove_stop_words(t1,it_stop_words)
#t1


In [5]:
def stemming(text):
    from nltk.stem import SnowballStemmer
    stemmer = SnowballStemmer('italian')
    return [stemmer.stem(word) for word in text]
#t1_stem

In [6]:
proc_letters = []
for letter in ita_corpus['text']:
    text = letter.lower()
    
    text = adv_tokenizer(text)
    text = remove_stop_words(text,it_stop_words)
    text = stemming(text)
    proc_letters.append(text)
    

In [7]:
ita_corpus['proc_text'] = proc_letters

In [8]:
ita_corpus.head()

,level_0,n,corpus,index,date,year,sender,senderLocation,recipient,recipientLocation,languages,mainLanguage,text,proc_text
0,4,4,Schmitz Joyce,5,05/01/1921,1921,James Joyce,Parigi,Ettore Schmitz,Trieste,"ITA,UND",ITA,Boulevard Raspail 5 Parigi VII Caro signor Sch...,"[boulevard, raspail, 5, parig, vii, car, signo..."
1,5,5,Schmitz Joyce,6,30/01/1924,1924,James Joyce,Parigi,Ettore Schmitz,Trieste,"ITA,FRE",ITA,Victoria Palace Hotel 6 rue Blaise des Gojjes ...,"[victor, palac, hotel, 6, rue, blais, des, goj..."
2,6,6,Schmitz Joyce,7,20/11/1924,1924,James Joyce,Parigi,Ettore Schmitz,Trieste,"ITA,LAT,FRE",ITA,Victoria Palace Hotel 6 rue Blaise des Goff...,"[victor, palac, hotel, 6, rue, blais, des, gof..."
3,7,7,Schmitz Joyce,8,01/04/1924,1924,James Joyce,Parigi,Ettore Schmitz,Trieste,"ITA,FRE",ITA,Restaurant des Trianons\t Téléphone Ségur 30-6...,"[restaurant, des, trianons, tèlèphon, sègur, 3..."
4,8,8,Schmitz Joyce,9,10/06/1924,1924,Ettore Schmitz,Trieste,James Joyce,Parigi,"ITA,ENG",ITA,Gioachino Veneziani Telefono n. 152 Indirizzo ...,"[gioachin, venezian, telef, n, 152, indirizz, ..."


In [14]:
import gensim
dictionary = gensim.corpora.Dictionary(proc_letters)
dictionary

In [15]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [17]:
bow = [dictionary.doc2bow(doc) for doc in proc_letters]

[(3, 1),
 (4, 2),
 (5, 1),
 (8, 1),
 (24, 1),
 (27, 3),
 (31, 2),
 (39, 1),
 (49, 1),
 (50, 1),
 (56, 4),
 (62, 1),
 (63, 2),
 (69, 2),
 (74, 1),
 (78, 1),
 (101, 2),
 (104, 1),
 (111, 1),
 (112, 1),
 (117, 1),
 (119, 1),
 (120, 1),
 (125, 1),
 (128, 1),
 (129, 1),
 (138, 1),
 (139, 1),
 (140, 1),
 (141, 1),
 (142, 1),
 (143, 1),
 (144, 1),
 (145, 1),
 (146, 1),
 (147, 1),
 (148, 1),
 (149, 1),
 (150, 1),
 (151, 1),
 (152, 1),
 (153, 1),
 (154, 1),
 (155, 1),
 (156, 1),
 (157, 1),
 (158, 1),
 (159, 1),
 (160, 1),
 (161, 1),
 (162, 1),
 (163, 1),
 (164, 1),
 (165, 1),
 (166, 1),
 (167, 1),
 (168, 1),
 (169, 1),
 (170, 1),
 (171, 1),
 (172, 1),
 (173, 1),
 (174, 1),
 (175, 1),
 (176, 1),
 (177, 1),
 (178, 1),
 (179, 3),
 (180, 1),
 (181, 2),
 (182, 2),
 (183, 1),
 (184, 1),
 (185, 1),
 (186, 1),
 (187, 1),
 (188, 1),
 (189, 2),
 (190, 3),
 (191, 1),
 (192, 1),
 (193, 1),
 (194, 1),
 (195, 1),
 (196, 1),
 (197, 1),
 (198, 1),
 (199, 1),
 (200, 2),
 (201, 2)]